### Connexion à MyGene.Info 

Cette ligne est utilisée pour initialiser un objet qui permet d'interagir avec la base de données de gènes en utilisant l'API de [MyGene.info](https://mygene.info/). Ici, une instance de la classe MyGeneInfo est créée et assignée à la variable ```mg```. Cette instance sera utilisée pour effectuer des requêtes à la base de données de gènes via l'API MyGene.info.

En résumé, cette ligne crée une connexion à la base de données de gènes MyGene.info, permettant au script d'interroger et de récupérer des informations sur les gènes et protéines spécifiés.

In [ ]:
mg = mygene.MyGeneInfo()

La fonction chercher_proteine(nom_proteine) prend en argument le nom d'une protéine et utilise l'objet ```mg``` (qui est une instance de ```mygene.MyGeneInfo()```) pour interroger la base de données de gènes via l'API MyGene.info. Elle récupère ensuite les termes GO associés à la composante cellulaire (CC) de la protéine.

In [ ]:
def chercher_proteine(nom_proteine):
    '''
    Cette ligne effectue une requête à la base de données de gènes en utilisant le nom de la protéine (nom_proteine). 
    La requête est spécifiée pour retourner uniquement les informations sur la composante cellulaire (CC) des annotations GO (fields="go.CC.id"), 
    pour l'espèce humaine (species="human"), et la taille de la réponse est limitée à 1 (size=1).
    '''
    r = mg.query('name:"%s"' % nom_proteine, fields = "go.CC.id",species = "human", size = 1)
    
    '''
    Si la réponse ne contient pas d'informations GO (indiquées par la clé "go" dans la première entrée des résultats), 
    la fonction retourne un ensemble vide. 
    '''
    if not "go" in r["hits"][0]: return set()
  
    '''
    Les informations GO de la composante cellulaire sont extraites de la réponse. 
    Si ces informations ne sont pas sous forme de liste, elles sont converties en liste.
    '''
    cc = r["hits"][0]["go"]["CC"]
    if not isinstance(cc, list): cc = [cc]
  
    '''
    La fonction itère à travers chaque élément de la liste d'annotations GO de la composante cellulaire. 
    Pour chaque élément, elle extrait l'identifiant GO (id_go), le convertit en un terme de l'ontologie GO (terme_go) 
    en utilisant l'objet obo, et ajoute ce terme à l'ensemble sites. Si le terme GO n'est pas trouvé dans l'ontologie, il est ignoré.
    '''
    sites = set()
    for dico in cc:
        id_go = dico["id"]
        terme_go = obo[id_go.replace(":", "_")]
        if terme_go: sites.add(terme_go)
    
    return sites

La fonction ``intersection_semantique(sites1, sites2)`` prend deux ensembles de sites GO (représentant les composantes cellulaires de deux protéines) en entrée et retourne l'ensemble des termes GO qui sont communs entre ces deux ensembles, tout en éliminant les sous-parties redondantes.

In [ ]:
def intersection_semantique(sites1, sites2):
    '''
    Ces boucles itèrent sur chaque site dans les ensembles sites1 et sites2 respectivement. 
    Pour chaque site, elle récupère toutes les sous-parties transitives de ce site et 
    les ajoute à l'ensemble correspondant (sous_parties1 ou sous_parties2).
    '''
    sous_parties1 = set()
    for site in sites1:
        sous_parties1.update(site.sous_parties_transitives())
    
    sous_parties2 = set()
    for site in sites2:
        sous_parties2.update(site.sous_parties_transitives())
    
    '''
    Cette ligne calcule l'intersection des ensembles de sous-parties transitives des deux ensembles de sites.
    '''
    sites_communs = sous_parties1 & sous_parties2
    
    '''
    Un dictionnaire cache est créé pour stocker les sous-parties transitives de chaque site commun. 
    Cela est fait pour éviter de recalculer les sous-parties transitives plusieurs fois.
    '''
    cache = { site : site.sous_parties_transitives()for site in sites_communs }
  
    '''
    Cette double boucle vérifie pour chaque paire de sites communs s'il existe une relation d'inclusion
    (une est une sous-partie de l'autre) dans le cache. Si tel est le cas, la sous-partie est considérée
    comme redondante et ignorée. L'ensemble résultant, sites_communs_sans_sous_parties, contient les 
    termes GO communs sans les sous-parties redondantes.
    '''
    sites_communs_sans_sous_parties = set()
    for site in sites_communs:
        for site2 in sites_communs:
            if (not site2 is site) and (site in cache[site2]): break
        else:
            sites_communs_sans_sous_parties.add(site)
  
    return sites_communs_sans_sous_parties

### Création d'une page web

Ce script Flask crée une interface web simple permettant à l'utilisateur de saisir le nom de deux protéines. La route "/resultat" utilise les fonctions `chercher_proteine` et `intersection_semantique` pour récupérer les sites GO associés à chaque protéine et affiche les résultats sur une page web. La page de saisie ("/") génère un formulaire avec deux champs de saisie. Lors de la soumission, l'utilisateur est redirigé vers la page de résultats, où les sites GO associés à chaque protéine et les sites GO communs sont affichés dans une page HTML triée et lisible. L'application Flask est exécutée sur le port 5000 du serveur local.

In [ ]:
@app.route('/')
def page_saisie():
    html  = """
<html><body>
  <form action="/resultat">
    Protéine 1: <input type="text" name="prot1"/><br/><br/>
    Protéine 2: <input type="text" name="prot2"/><br/><br/>
    <input type="submit"/>
  </form>
</body></html>"""
    return html
  
@app.route('/resultat')
def page_resultat():
    prot1 = request.args.get("prot1", "")
    prot2 = request.args.get("prot2", "")
  
    sites1 = chercher_proteine(prot1)
    sites2 = chercher_proteine(prot2)
  
    sites_communs = intersection_semantique(sites1, sites2)
  
    html  = """<html><body>"""
    html += """<h3>Site de la protéine 1 (%s)</h3>""" % prot1
    if sites1:
        html += "<br/>".join(sorted(str(site) for site in sites1))
    else:
        html += "(Aucun)<br/>"
  
    html += """<h3>Site de la protéine 2 (%s)</h3>""" % prot2
    if sites2:
        html += "<br/>".join(sorted(str(site) for site in sites2))
    else:
        html += "(Aucun)<br/>"
  
    html += """<h3>Sites de rencontre possibles</h3>"""
    if sites_communs:
        html += "<br/>".join(sorted(str(site) for site in sites_communs))
    else:
        html += "(Aucun)<br/>"
    
    html += """</body></html>"""
    return html
  
import werkzeug.serving
werkzeug.serving.run_simple("localhost", 5000, app)